<a href="https://colab.research.google.com/github/RafsanJany-44/ARC_5SSC_Experiment/blob/main/SleepStage_EEG_EXperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Function

In [19]:
!pip install imbalanced-learn
!pip install ipywidgets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [21]:
def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)

def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic


def models_check_box(models):
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys


def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)


def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

#Starting

In [22]:
dataset = pd.read_csv("/content/drive/MyDrive/Sleep Stage 5 class/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

(108920, 52)
(108450, 52)


###Spliting into X and y

In [23]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]

#Feature Selection

In [24]:
number_of_feat = 20

###ANOVA with f classifciation

In [25]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "ANOVA"

###Embedded Method

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
selection_method = "Embedded"

###Pearson's with f regression

In [ ]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(dataset.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "Pearson"

###Sequential Feature Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]

selection_method = "Sequential"

###Feature list

In [27]:
X=dataset[list_of_feat]
y=dataset[target]
print(X.shape)

(108450, 20)


#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [42]:
def tune_1():
  X_new = X
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [43]:
def tune_2():
  X_new = StandardScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [44]:
def tune_3():
  X_new = MinMaxScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [45]:
def tune_4():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [46]:

def tune_5():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [47]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [48]:

def tune_7():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [49]:
def tune_8():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [50]:
def tune_9():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [51]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [52]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [53]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_new = StandardScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : All
##Scaling : ALL (MinMax)

In [54]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_new = MinMaxScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [55]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  #"Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  #"Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  #"Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  #"Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  #"Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  #"Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(MinMax)"        : tune_13()
}



---



# **Training**

##ADABOOST

In [56]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada_defult = AdaBoostClassifier(random_state=0)

ada={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  ada_defult.fit(X_train, y_train)
  y_pred = ada_defult.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  ada[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(ada, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_ada.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.5631627478100507
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.5631627478100507
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.5631627478100507
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.534822968448818
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5142461964038727
--------------------------------------------



---



---



##Graddient Boosting

In [57]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gradBoost_default = GradientBoostingClassifier(random_state=0)

grad={}

for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  gradBoost_default.fit(X_train, y_train)
  y_pred = gradBoost_default.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  grad[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(grad, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_grad.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6349008759797141
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6349008759797141
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6349008759797141
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.6185705589936086
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5937759336099585
-------------------------------------------

###Histogram-Based Gradient Boosting

In [58]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)

hisgrad={}

for i in list(tune_dic.keys()):
  X_train, X_test, y_train, y_test = tune_dic[i]
  
  hisgradBoost_default.fit(X_train, y_train)
  y_pred = hisgradBoost_default.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  hisgrad[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(hisgrad, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_hisgrad.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6808667588750577
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6808667588750577
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6808667588750577
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.6971441615095871
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.6485477178423237
-------------------------------------------



---



---



##Random Forest

In [59]:
%%time
RanFor={}

from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)


for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  rf_default.fit(X_train, y_train)
  y_pred=rf_default.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  RanFor[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(RanFor, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_RanFora.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6940525587828492
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6940064545873674
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6937759336099585
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.8349142741199148
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.6823420931304749
-------------------------------------------



---



---



##XGB

In [60]:
%%time
import xgboost as xgb
li = ["Smote : NO    ; Scaling : Train(Standard)",
      "Smote : NO    ; Scaling : Train(MinMax)",
      "Smote : Train ; Scaling : Train(Standard)",
      "Smote : Train ; Scaling : Train(MinMax)",
      "Smote : All   ; Scaling : Train(Standard)",
      "Smote : All   ; Scaling : Train(MinMax)"]
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)

xgb={}

for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]

  if i in li:
    xgb_deafult.fit(X_train,y_train.values)
    y_pred = xgb_deafult.predict(X_test.values)
  else:
    xgb_deafult.fit(X_train,y_train)
    y_pred = xgb_deafult.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  xgb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(xgb, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_xgb.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6145689257722453
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6145689257722453
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6145689257722453
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.5973673531500456
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5698017519594283
-------------------------------------------

##KNN

In [61]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)

knn={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  knn_default.fit(X_train, y_train)
  y_pred=knn_default.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  knn[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(knn, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_knn.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.60567081604426
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.629045643153527
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6205624711848778
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.7524601805823273
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5510373443983403
----------------------------------------------

##NB

In [62]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()

nb={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  nb_deafult.fit(X_train, y_train)
  y_pred = nb_deafult.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  nb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(nb, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_nb.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.44430613185799905
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.44430613185799905
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.44430613185799905
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.44054986304149335
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.4219917012448133
---------------------------------------

##Support Vector Machines

####SVM

In [ ]:
"""%%time
from sklearn import svm

svm = svm.SVC(decision_function_shape='ovo')

svm_svc={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  svm.fit(X_train, y_train)
  y_pred = svm.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  svm_svc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(svm_svc, orient="index")
df.to_csv("svm_svc.csv")"""

####Linear

In [ ]:
"""%%time
from sklearn import svm

LinearSVC = svm.LinearSVC()

lin_svc = {}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  LinearSVC.fit(X_train, y_train)
  y_pred = LinearSVC.predict(X_test)



  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  lin_svc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(lin_svc, orient="index")
df.to_csv("lin_svc.csv")"""

##Stochastic Gradient Descent

In [ ]:
"""%%time
from sklearn.linear_model import SGDClassifier

SGDClassifier = SGDClassifier()

sgdc={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  SGDClassifier.fit(X_train, y_train)
  y_pred = SGDClassifier.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  sgdc[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(sgdc, orient="index")
df.to_csv("sgdc.csv")"""

##Tree algorithms

###DecisionTreeRegressor

In [63]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

dt={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  dtc.fit(X_train, y_train)
  y_pred = dtc.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  dt[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(dt, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_dt.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.5761641309359151
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.575749193176579
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.5746426924850161
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.7064522674241656
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.5542185338865837
--------------------------------------------

In [64]:
!pip install catboost

from catboost import CatBoostClassifier

#cat = CatBoostClassifier(task_type="GPU")

cat=CatBoostClassifier()

catb={}
for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]
  cat.fit(X_train,y_train)
  y_pred = cat.predict(X_test)

  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  catb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(catb, orient="index")
df.to_csv(selection_method+"_"+str(number_of_feat)+"_features_catb.csv")

Streaming output truncated to the last 5000 lines.
24:	learn: 1.0578347	total: 2.42s	remaining: 1m 34s
25:	learn: 1.0529556	total: 2.52s	remaining: 1m 34s
26:	learn: 1.0485444	total: 2.61s	remaining: 1m 33s
27:	learn: 1.0448397	total: 2.69s	remaining: 1m 33s
28:	learn: 1.0413670	total: 2.77s	remaining: 1m 32s
29:	learn: 1.0366353	total: 2.86s	remaining: 1m 32s
30:	learn: 1.0326762	total: 2.95s	remaining: 1m 32s
31:	learn: 1.0287979	total: 3.04s	remaining: 1m 32s
32:	learn: 1.0256839	total: 3.13s	remaining: 1m 31s
33:	learn: 1.0224073	total: 3.23s	remaining: 1m 31s
34:	learn: 1.0195541	total: 3.32s	remaining: 1m 31s
35:	learn: 1.0165191	total: 3.41s	remaining: 1m 31s
36:	learn: 1.0135152	total: 3.5s	remaining: 1m 31s
37:	learn: 1.0108368	total: 3.6s	remaining: 1m 31s
38:	learn: 1.0078228	total: 3.69s	remaining: 1m 30s
39:	learn: 1.0049059	total: 3.79s	remaining: 1m 30s
40:	learn: 1.0027025	total: 3.88s	remaining: 1m 30s
41:	learn: 0.9995189	total: 3.98s	remaining: 1m 30s
42:	learn: 0.99



---



---

